<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_sm
!python -m spacy download ru_core_news_md

2021-04-11 11:52:01.186925: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 17.9 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
2021-04-11 11:52:15.197336: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 43.9 MB 20 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [4]:
pip install pymorphy2[fast]

In [5]:
pip install Gensim

In [6]:
import spacy
import gensim
from gensim.models import Word2Vec 
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd

In [7]:
nlp = spacy.load('ru_core_news_md')

In [8]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt

--2021-04-11 11:52:40--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3638613 (3.5M) [text/plain]
Saving to: ‘neuroleptic_ru.txt.1’

neuroleptic_ru.txt. 100%[===================>]   3.47M  --.-KB/s    in 0.1s    

2021-04-11 11:52:41 (29.9 MB/s) - ‘neuroleptic_ru.txt.1’ saved [3638613/3638613]



In [9]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-04-11 11:52:41--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz.1’

news_upos_cbow_300_ 100%[===================>] 133.81M   189MB/s    in 0.7s    

2021-04-11 11:52:42 (189 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz.1’ saved [140310878/140310878]



In [10]:
handler = open('neuroleptic_ru.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

In [11]:
posts = str_list = my_txt.split('----')

In [12]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

In [13]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('news_upos_cbow_300_2_2017.bin.gz', binary=True)

In [14]:
model = Word2Vec(size=300, min_count=1)
model.build_vocab(sentences_tokenized)
total_examples = model.corpus_count
model.build_vocab([list(w2v.vocab.keys())], update=True)
model.intersect_word2vec_format("news_upos_cbow_300_2_2017.bin.gz", binary=True)
model.train(sentences_tokenized, total_examples=total_examples, epochs=model.iter)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


(1321515, 1530870)

In [15]:
model.most_similar('препарат')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('который', 0.9999386072158813),
 ('когда', 0.9998950958251953),
 ('даже', 0.9998393058776855),
 ('легче', 0.9998388886451721),
 ('эффект', 0.9998361468315125),
 ('можно', 0.9998328685760498),
 ('жизни', 0.9998270273208618),
 ('к', 0.9998190402984619),
 ('еще', 0.999817430973053),
 ('гораздо', 0.9997991919517517)]

In [16]:
# model['алимемазин']
model.wv.most_similar('алимемазин')

[('врачом', 0.9961577653884888),
 ('собой', 0.9961369633674622),
 ('весьма', 0.9960938096046448),
 ('во', 0.9960758686065674),
 ('между', 0.996065616607666),
 ('советы', 0.9960624575614929),
 ('написать', 0.9960455894470215),
 ('при', 0.9960418939590454),
 ('без', 0.9960407018661499),
 ('под', 0.9959831833839417)]

In [17]:
cosine_similarity(np.atleast_2d(model.wv.get_vector('алимемазин')), \
                  np.atleast_2d(model.wv.get_vector('азалептин')))[0][0]

0.9940834

In [18]:
cosine_similarity(np.atleast_2d(model.wv.get_vector('алимемазин')), \
                  np.atleast_2d(model.wv.get_vector('стул')))[0][0]

0.876649

In [19]:
posts[9]

'\nПринимал порядка 2-3 недель от 10 до 20 мг Алимемазина, совместно с 200 мг Азалептина и 30 мг Ремерона.Всё это принималось за полчаса до сна.Лично я не почувствовал действие Тералиджена, словно и не принимал его вовсе.Мою довольно средневзвешенную ипохондрию он не купировал,равно как и депрессивные симптомы.\n'

**Нужно ли мне сначала извлекать названия лекарств?**

In [20]:
posts[0]

'На рынок снова вернулся алимемазин (раньше терален назывался). Хотелось бы узнать Ваше мнение. Кто принимал в своей практике, что хорошего, что плохого\n\n____________________________________________\nСообщение отредактировано.\n\nДобавлена ссылка на ролик в You Tube.\n\nСообщение отредактировал Гилев: 12 Апрель 2020 - 07:55\nДобавлена ссылка на ролик в You Tube\n'

In [21]:
doc = nlp(u'Кто принимал его в своей практике? Это не хороший препарат.')

In [22]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [23]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_, 
            [child for child in token.children], morph.parse(token.text)[0].tag)


Кто nsubj принимал VERB [] NPRO,masc sing,nomn
принимал ROOT принимал VERB [Кто, его, практике, ?] VERB,impf,tran masc,sing,past,indc
его obj принимал VERB [] NPRO,masc,3per,Anph sing,accs
в case практике NOUN [] PREP
своей det практике NOUN [] ADJF,Apro,Anph femn,sing,gent
практике obl принимал VERB [в, своей] NOUN,inan,femn sing,loct
? punct принимал VERB [] PNCT
Это nsubj препарат NOUN [] PRCL
не advmod хороший ADJ [] PRCL
хороший amod препарат NOUN [не] ADJF,Qual masc,sing,nomn
препарат ROOT препарат NOUN [Это, хороший, .] NOUN,inan,masc sing,nomn
. punct препарат NOUN [] PNCT


In [24]:
def find_args(my_text, my_spacy):
  sents = text.split('.')
  for sent in sents:
    doc = my_spacy(u'Кто принимал его в своей практике')

In [25]:
token_list = [i for i in doc]

In [26]:
for token in doc:
  if token.dep_ == 'obj':
    print(token.text, token.dep_, token.head.text, token.head.pos_,
        [child for child in token.children])

его obj принимал VERB []


In [27]:
sentences_tokenized[2]

['в',
 'волгограде',
 'есть',
 'почти',
 'во',
 'всех',
 'аптеках',
 'стоит',
 '80',
 'руб',
 'мы',
 'его',
 'применяемм',
 'в',
 'основном',
 'как',
 'вегетостабилизатор']

In [71]:
sentences_for_annts = [' '.join(sent) for sent in sentences_tokenized]
sentences_for_annts = [sent.split() for sent in set(sentences_for_annts)]
sents_for_annts= []
for sent in sentences_for_annts:
  sent.append('\n')
  sents_for_annts.extend(sent) 

In [72]:
len(sents_for_annts)

233133

In [73]:
df = pd.DataFrame(columns=['TOKEN', 'NER', 'COREFERENCE_CLUSTER'])

In [74]:
df['TOKEN'] = sents_for_annts
df['NER'] = [0 for i in range(len(sents_for_annts))]

In [75]:
df

,TOKEN,NER,COREFERENCE_CLUSTER
0,\n,0,NaN
1,написал,0,NaN
2,в,0,NaN
3,личку,0,NaN
4,\n,0,NaN
...,...,...,...
233128,препарат,0,NaN
233129,мы,0,NaN
233130,не,0,NaN
233131,диетологи,0,NaN


In [76]:
df.to_csv('for_annts.csv', sep=' ', index=False)

In [77]:
from google.colab import files


In [78]:
files.download('for_annts.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>